In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import collections
import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sherlock.db'
mask_file = '/home/eli/Documents/sherlock/movie_files/wholebrain.nii.gz'

OVERWRITE = True

MOVIE_RUN_LENGTHS = [946, 1030]
MOVIE_1_SCENES = '''Scene Onset Offset
     1     1    26
     2    27    35
     3    36    55
     4    56    86
     5    88   109
     6   110   130
     7   132   143
     8   145   157
     9   158   172
    10   174   192
    11   194   203
    12   206   225
    13   227   313
    14   315   361
    15   364   399
    16   400   503
    17   508   525
    18   527   534
    19   536   615
    20   616   633
    21   635   641
    22   643   679
    23   680   697
    24   700   747
    25   749   868
    26   873   891
    27   892   945
'''

MOVIE_2_SCENES = '''Scene Onset Offset
1           1          25
2          27          69
3          71         129
4         131         145
5         146         193
6         195         275
7         276         325
8         344         412
9         414         489
10         491         523
11         525         635
12         636         671
13         672         728
14         730         747
15         748         756
16         758         790
17         791         813
18         815         859
19         860         875
20         877         912
21         914         960
22         962        1000
23        1001        1030
'''

In [4]:
bad_scenes = collections.defaultdict(lambda: collections.defaultdict(lambda: set()))
bad_scenes[5][1] = set([23])

In [5]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

if not existed or OVERWRITE:
    data_files = glob.glob('/home/eli/Documents/sherlock/movie_files/*.nii')
    regex = re.compile("sherlock_movie_s([0-9]{1,2}).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        subject = int(m.groups()[0])

        tsv_reader = csv.DictReader(MOVIE_1_SCENES.splitlines(), delimiter=' ', skipinitialspace=True)
        scenes = [row for row in tsv_reader]
        for (scene, onset, offset) in [s.values() for s in scenes]:
            if scene in bad_scenes[subject][0]:
                logging.info('Excluding scene %d of Movie 1 for subject %d for flat data',
                             scene, subject)
                continue
            block = niidb.FMriActivationBlock(zscore=False)
            block.filename = data_file
            block.subject = subject
            block.run = 1
            block.task = 'sherlock_movie1_scene%d' % int(scene)
            block.start_time = int(onset) - 1
            block.end_time = int(offset)
            db.upsert(block)
            logging.info("Scene %d of subject %d's first run: from %d to %d", int(scene), subject, block.start_time, block.end_time)
        
        tsv_reader = csv.DictReader(MOVIE_2_SCENES.splitlines(), delimiter=' ', skipinitialspace=True)
        scenes = [row for row in tsv_reader]
        for (scene, onset, offset) in [s.values() for s in scenes]:
            scene = int(scene)
            if scene in bad_scenes[subject][1]:
                logging.info('Excluding scene %d of Movie 1 for subject %d for flat data',
                             scene, subject)
                continue
            onset = int(onset) + MOVIE_RUN_LENGTHS[0]
            offset = int(offset) + MOVIE_RUN_LENGTHS[0]
            block = niidb.FMriActivationBlock(zscore=False)
            block.filename = data_file
            block.subject = subject
            block.run = 2
            block.task = 'sherlock_movie2_scene%d' % scene
            block.start_time = int(onset) - 1
            block.end_time = int(offset)
            db.upsert(block)
            logging.info("Scene %d of subject %d's second run: from %d to %d", int(scene), subject, block.start_time, block.end_time)


10/02/2018 21:39:19 Scene 1 of subject 13's first run: from 0 to 26
10/02/2018 21:39:19 Scene 2 of subject 13's first run: from 26 to 35
10/02/2018 21:39:19 Scene 3 of subject 13's first run: from 35 to 55
10/02/2018 21:39:19 Scene 4 of subject 13's first run: from 55 to 86
10/02/2018 21:39:19 Scene 5 of subject 13's first run: from 87 to 109
10/02/2018 21:39:19 Scene 6 of subject 13's first run: from 109 to 130
10/02/2018 21:39:19 Scene 7 of subject 13's first run: from 131 to 143
10/02/2018 21:39:19 Scene 8 of subject 13's first run: from 144 to 157
10/02/2018 21:39:19 Scene 9 of subject 13's first run: from 157 to 172
10/02/2018 21:39:19 Scene 10 of subject 13's first run: from 173 to 192
10/02/2018 21:39:19 Scene 11 of subject 13's first run: from 193 to 203
10/02/2018 21:39:19 Scene 12 of subject 13's first run: from 205 to 225
10/02/2018 21:39:19 Scene 13 of subject 13's first run: from 226 to 313
10/02/2018 21:39:19 Scene 14 of subject 13's first run: from 314 to 361
10/02/2018 

In [6]:
logging.info('Finished building NiiDb out of Sherlock_Merlin dataset')

10/02/2018 21:40:06 Finished building NiiDb out of Sherlock_Merlin dataset
